In [1]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
except:
    print('Something went wrong...')

In [4]:
########## Function to Send Email #########

def sendmail(sub, body):
    try:
       SERVER = "us.relay.ibm.com"
       #SERVER = "smtp.gmail.com"
       #SERVER = smtplib.SMTP('smtp.gmail.com', 587)
       #SERVER.ehlo()
       FROM = "romdbload@radial.com"
       TO = ["mlakshm@us.ibm.com"] # must be a list
       SUBJECT = sub
       TEXT = body
       message = 'From:'+FROM+\
       '\nTo: '+", ".join(TO)+\
       '\nSubject: '+SUBJECT+'\n'+TEXT
       print(message)
       server = smtplib.SMTP(SERVER)
       server.sendmail(FROM, TO, message)
       server.quit()
       return 0
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)


In [5]:
sendmail("Test","Test")

From:romdbload@radial.com
To: mlakshm@us.ibm.com
Subject: Test
Test


0

In [6]:
tbllist=[]
jsonpathlists=[]
colslist=[]
json_exploded_path_lists=[]

with open('json_column_map_updated.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        json_exploded_path_list=[]
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==3):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
               json_exploded_path_list.append(coljsonarr[2])
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
        json_exploded_path_lists.append(json_exploded_path_list)
            
            
f.close()


tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]
json_exploded_path_lists = [x for x in json_exploded_path_lists if x != []]

print(tbllist[6])
print(jsonpathlists[6])
print(colslist[6])
print(json_exploded_path_lists[6])

KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0
['order.lineItems.0.charges.0.taxes.0.jurisdictionLevel', 'order.id', 'order.lineItems.0.charges.0.taxes.0.situs', 'order.lineItems.0.charges.0.taxes.0.impositionValue', 'order.sellerId', 'order.lineItems.0.charges.0.taxes.0.impositionType', 'order.lineItems.0.charges.0.taxes.0.jurisdictionId']
['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']
['order_lineItems_charges_taxes.jurisdictionLevel', 'order.id', 'order_lineItems_charges_taxes.situs', 'order_lineItems_charges_taxes.impositionValue', 'order.sellerId', 'order_lineItems_charges_taxes.impositionType', 'order_lineItems_charges_taxes.jurisdictionId']


In [7]:

##### Input Tables ######

#tbllist=['KAFKARADIAL.ORDER_HEADER_STG', 'KAFKARADIAL.ORDER_LINE_STG', 'KAFKARADIAL.ORDER_CHARGES_STG']

In [8]:

##### Column Mapping ######

#jsonpathlists=[['msg.order.orderTotal.promotionCharge', 'msg.order.id', 'msg.order.sellerId', 'msg.order.orderTotal.tax', 'msg.order.orderTotal.total', 'msg.order.orderTotal.adjustmentCharge', 'msg.order.type'], ['msg.order.id', 'msg.order.lineItems.0.quantity', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.color', 'msg.order.lineItems.0.lineTotal.totalTaxAmount', 'msg.order.lineItems.0.unitPrice', 'msg.order.lineItems.0.lineTotal.totalPromotionChargeAmount', 'msg.order.lineItems.0.lineTotal.netChargeAmount', 'msg.order.lineItems.0.itemDetail.itemId', 'msg.order.lineItems.0.lineTotal.totalAdjustmentChargeAmount', 'msg.order.lineItems.0.itemDetail.itemDesc', 'lookup'], ['msg.order.id', 'msg.order.lineItems.0.charges.0.id', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.charges.0.detail.amount', 'msg.order.lineItems.0.charges.0.name']]

In [9]:

###### Columns of the Tables #####

#colslist=[['ORDER_PROMO_AMT', 'ORDER_ID', 'SELLER_ID', 'ORDER_TAX_AMT', 'ORDER_TOTAL_AMT', 'ORDER_ADJ_AMT', 'ORDER_TYPE'], ['ORDER_ID', 'ITEM_QTY', 'LINE_NUM', 'COLOR', 'LINE_TAX_AMT', 'ITEM_AMT', 'LINE_PROMO_AMT', 'LINE_TOTAL_AMT', 'ITEM_ID', 'LINE_ADJ_AMT', 'ITEM_DESC', 'COUNTRY_OF_ORIGIN'], ['ORDER_ID', 'CHARGE_ID', 'LINE_NUM', 'CHARGE_AMOUNT', 'CHARGE_NAME']]

In [10]:

########## Reading Input Stream in JSON Format from Kafka ###########

try:

 df = spark \
  .readStream \
  .option("multiLine", True).option("mode", "PERMISSIVE") \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test3") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)
    

In [11]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
###WASHINGTON

jdbcUrl="jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;"
user="a5406f1c"
password="eK1EhADAU7y3W61T"

In [13]:
###### Lookup Country of Origin #######

key ="'100'"
keyval="SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID="+key

print(keyval)
dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+keyval+') as t').option('user', user).option('password', password).load()
dbl.show()


SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID='100'
+-----------------+
|COUNTRY_OF_ORIGIN|
+-----------------+
+-----------------+



In [14]:
def writeToSQLWarehouse(df,epochId,tbl):
  print(tbl)
  #print(df)
  df.write.format("jdbc") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("dbtable", tbl) \
  .option("user", user) \
  .option("password", password) \
  .save()

In [15]:
def prepMultiRows(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       dfr = spark.read.option("multiline", "true") \
       .json(spark.sparkContext.parallelize([x]))
       dfr.show()
       l.append(dfr)
    print("***")

    return l

In [16]:
##### Exploding JSON Arrays Into Columns #####

def explodeDF(df):
    
    arraysdf = pd.read_csv("arrays_order_created.txt", sep=",")
    for i, row in arraysdf.iterrows():
    
        json_path_to_explode=row['JSON_PATH_TO_EXPLODE']
        json_path_exploded=row['JSON_PATH_EXPLODED']
    
        df=df.withColumn(json_path_exploded,explode(json_path_to_explode))
        df.show()
        
    return df

In [17]:
def popTablesBlk(df, epochId):  #### Bulk Insert
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochId)
    print("after")
    
    # Create an empty RDD
    emp_RDD = spark.sparkContext.emptyRDD()
 
    # Create empty schema
    columns = StructType([])
 
    # Create an empty RDD with empty schema
    dfblk = spark.createDataFrame(data = emp_RDD, schema = columns)
    
    i=0
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            if i==0:
                dfblk=jmsg
                i=i+1
            else:
                dfblk.union(jmsg)  
            
        else:
            print("Corrupt Record:")
            jmsg.show()
            
    prepareExpDF(dfblk,epochId)
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)


In [18]:
def popTables(df, epochId):
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    dflist=prepMultiRows(df,epochId)
    print("after")
    for jmsg in dflist:
        jmsg.show()
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            jmsg=explodeDF(jmsg)
            prepareDF(jmsg,json_exploded_path_lists,colslist,epochId,tbllist)
        else:
            print("Corrupt Record:")
            jmsg.show()
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)



In [19]:
lookuptablelist=[["KAFKARADIAL.PRODUCT_MASTER","COUNTRY_OF_ORIGIN"]]

In [20]:
###### Prepare Dataframe and Populate the DB #######

## [[PRODUCT_MASTER,COUNTRY_OF_ORIGIN],[<table2>,<field2>]]

def lookupItemID(lookuptablelist, item_id):
    
    try:
       
     for lookuptable in lookuptablelist:
            
            lookuptblname=lookuptable[0]
            lookuptblfield=lookuptable[1]
            item_id="'"+item_id+"'"
            print(item_id)
            qry='SELECT '+lookuptblfield+' FROM '+lookuptblname+' WHERE ITEM_ID = '+item_id
            print(qry)
            dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+qry+') as t').option('user', user).option('password', password).load()
            dbl.show()
            ctry=dbl.collect()[0][0]
            return ctry

       #### Create Dataframe with JSON value list parsed as the data and the Table Column Names as the Schema ####
       #sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    
    
    except Exception as e:
        print("Exception:")
        print(e)
        exp_tb=traceback.format_exc()
        sendmail("ERROR",exp_tb)
        #print(exp_tb)

In [21]:
#ctry=lookupItemID(lookuptablelist,'100')

In [22]:
import builtins
round = getattr(builtins, "round")
from pyspark.sql.functions import lit

def updateDF(sdf,tbl):
    
    sdf.printSchema()
    if tbl=="KAFKARADIAL.ROM_ORDER_LINE_STG0":
       try:
        sdf=sdf.withColumn("IS_ASSOCIATE_DELIVERY", when(col("IS_ASSOCIATE_DELIVERY") == "false",lit("0") )\
        .when(col("IS_ASSOCIATE_DELIVERY") == "true",lit("1")) \
        .otherwise(col("IS_ASSOCIATE_DELIVERY")))
       except Exception as e:
         print(e)
    
    return sdf
    

In [23]:
###### Prepare Dataframe and Populate the DB #######

def prepareDF(df,json_exploded_path_lists,colslist,epochId,tbllist):
    
  for json_exploded_path_list,romlistfinal1,tbl in zip(json_exploded_path_lists,colslist,tbllist): 
    
    try:
         
       print("&&&&&&&&&&")
       df.show()
       jsonpathlist_clean=[]
       rom_list_clean = [] 
       for elm,colval in zip(json_exploded_path_list,romlistfinal1):
        
           try:
             df.select(elm)
             jsonpathlist_clean.append(elm)
             rom_list_clean.append(colval)  
           except Exception as e:
             print(e)
              
        
       
       cdf=df.select(jsonpathlist_clean)
       cdf.show()
            
       sdf=cdf.toDF(*rom_list_clean)
       sdf.show()
     
    
       ######## Update Dataframe ########
       print("Updating")
       sdf = updateDF(sdf,tbl)
       sdf.show()
    
    
       ##### Remove Duplicates ########
       print("Dropping Duplicates")
       finaldf=sdf.dropDuplicates(rom_list_clean)
       finaldf.show()

       ###### Insert Dataframe to DB #######
        
       writeToSQLWarehouse(finaldf, epochId,tbl)
    
    except Exception as e:
        print("hererrerre")
        print(e)

In [24]:
def pullCtry(jsonpathval,romlistwithval,colslist,lookupcolumn,lookupkey,tbl):
    
    ############# Pull COUNTRY_OF_ORIGIN ##############
    
    try:
    
      if ( lookupcolumn in colslist ):

          print("Inside*******")
        
          item_id_idx=romlistwithval.index(lookupkey)
          item_id=jsonpathval[item_id_idx]
          print(item_id)
          tableinfo=[tbl,lookupcolumn]
          lookuptablelist.append(tableinfo)
       
    
          ctry=lookupItemID(lookuptablelist, item_id)
          jsonpathval.append(ctry)
          romlistwithval.append(lookupcolumn)
     
    except Exception as e:
          print(e)
        
    
    return jsonpathval, romlistwithval
    
    

In [28]:
####### Writing Steams to DB #######

query1 = df \
    .writeStream \
    .outputMode("append") \
    .option("partition.assignment.strategy", "range")\
    .foreachBatch(popTables)\
    .start()

query1.awaitTermination(10)

<bound method RDD.isEmpty of MapPartitionsRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0>
before
+--------------------+
|               value|
+--------------------+
|{  "topicName" : ...|
+--------------------+

+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

***
after
+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

****
+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     o

False

+----------------+--------------------+--------------------+--------------------+-----------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_lineItems_charges|
+----------------+--------------------+--------------------+--------------------+-----------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|   {[], Price, [], {...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|   {[], Shipping, []...|
+----------------+--------------------+--------------------+--------------------+-----------------------+

+----------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_lineItems_charges|order_lineItems_charges_taxes|
+----------------+--------------------+--------------------+--------------------+----------

+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------------+
|   ORDER_ID|SIZE_CODE|SELLER_ID|COLOR|IS_ASSOCIATE_DELIVERY|  SIZE|COLOR_CODE|ITEM_UOM|      ITEM_ID|ITEM_GII|ORDER_LINE_KEY|
+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------------+
|30002217646|       MD|     浣熊|Black|                false|Medium|       001|    EACH|75-MH06-MD-BK|32306995|             1|
|30002217646|       MD|     浣熊|Black|                false|Medium|       001|    EACH|75-MH06-MD-BK|32306995|             1|
|30002217646|       MD|     浣熊|Black|                false|Medium|       001|    EACH|75-MH06-MD-BK|32306995|             1|
|30002217646|       MD|     浣熊|Black|                false|Medium|       001|    EACH|75-MH06-MD-BK|32306995|             1|
+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------

+-----------+--------+
|         id|sellerId|
+-----------+--------+
|30002217646|    浣熊|
|30002217646|    浣熊|
|30002217646|    浣熊|
|30002217646|    浣熊|
+-----------+--------+

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
+-----------+---------+

Updating
root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
+-----------+---------+

Dropping Duplicates
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
+-----------+---------+

KAFKARADIAL.ROM_ORDER_PAYMENT_STG0
&&&&&&&&&&
+----------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               ord

+------------------+-----------+-----------+--------------------+---------+--------------------+---------------+
|JURISDICTION_LEVEL|   ORDER_ID|  TAX_SITUS|    IMPOSITION_VALUE|SELLER_ID|     IMPOSITION_TYPE|JURISDICTION_ID|
+------------------+-----------+-----------+--------------------+---------+--------------------+---------------+
|            COUNTY|30002217646|DESTINATION|Local Sales and U...|     浣熊|General Sales and...|           5474|
|             STATE|30002217646|DESTINATION|   Sales and Use Tax|     浣熊|General Sales and...|           5286|
+------------------+-----------+-----------+--------------------+---------+--------------------+---------------+

KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0
&&&&&&&&&&
+----------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_lineItems_charges|order_lineItems_charges_taxes|
+--

+-----------+--------+-----+
|         id|sellerId| type|
+-----------+--------+-----+
|30002217646|    浣熊|SALES|
|30002217646|    浣熊|SALES|
|30002217646|    浣熊|SALES|
|30002217646|    浣熊|SALES|
+-----------+--------+-----+

+-----------+---------+----------+
|   ORDER_ID|SELLER_ID|ORDER_TYPE|
+-----------+---------+----------+
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
+-----------+---------+----------+

Updating
root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)
 |-- ORDER_TYPE: string (nullable = true)

+-----------+---------+----------+
|   ORDER_ID|SELLER_ID|ORDER_TYPE|
+-----------+---------+----------+
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
|30002217646|     浣熊|     SALES|
+-----------+---------+----------+

Dropping Duplicates
+-----------+---------+----------+
|   ORDER_ID|SELLER_ID|ORDER_TYPE|
+

In [29]:
query1.stop()

In [30]:
query1.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}